# Introduction

Welcome to this tutorial on using jupyter notebooks with Moveit 2. A great benefit of being able to interact with MoveIt via a Python notebook is the ability to rapidly prototype code. We hope you find this interface intuitive and that you gain value from using MoveIt via Python notebooks.

In this tutorial we will cover the following: 

* The required imports to run the notebook
* A motion planning example
* A teleoperation example

If you have suggestions or feedback for this tutorial please post an issue on GitHub (https://github.com/ros-planning/moveit2_tutorials) and tag @peterdavidfagan.

## Imports

Note: to launch this notebook and the nodes it depends on you must first specify a launch file. Details are provided earlier in this tutorial ().

In [1]:
import os
import math
import sys
import yaml
import rclpy
import numpy as np

# message libraries
from geometry_msgs.msg import PoseStamped, Pose

# moveit_py
from moveit.planning import MoveItPy
from moveit.core.robot_state import RobotState
from moveit.core.kinematic_constraints import construct_joint_constraint

# config file libraries
from moveit_configs_utils import MoveItConfigsBuilder
from ament_index_python.packages import get_package_share_directory


# we need to specify our moveit_py config at the top of each notebook we use. 
# this is since we will start spinning a moveit_py node within this notebook.

moveit_config = (
        MoveItConfigsBuilder(robot_name="lite6", package_name="moveit_resources_lite6_moveit_config")
        .robot_description_semantic(file_path="config/lite6.srdf")
        .trajectory_execution(file_path="config/moveit_controllers.yaml")
        .robot_description(file_path=get_package_share_directory("moveit_resources_lite6_description")
                          + "/urdf/lite6.urdf")
        .moveit_cpp(
            file_path=get_package_share_directory("lite6_moveit_demos")
            + "/config/jupyter_notebook_prototyping.yaml"
        )
        .to_moveit_configs()
    ).to_dict()

## Setup

In [2]:
# initialise rclpy (only for logging purposes)
rclpy.init()

# instantiate moveit_py instance and a planning component for the panda_arm
moveit = MoveItPy(node_name="moveit_py", config_dict=moveit_config)
lite6 = moveit.get_planning_component("lite6")

[INFO] [1676474926.256362222] [moveit_cpp_initializer]: Initialize rclcpp
[INFO] [1676474926.256423122] [moveit_cpp_initializer]: Initialize node parameters
[INFO] [1676474926.279231799] [moveit_cpp_initializer]: params_filepath: /tmp/launch_params_r8346a8w
[INFO] [1676474926.279243999] [moveit_cpp_initializer]: Initialize node and executor
[INFO] [1676474926.329649343] [moveit_cpp_initializer]: Spin separate thread
[INFO] [1676474926.331147439] [moveit_rdf_loader.rdf_loader]: Loaded robot model in 0.00136235 seconds
[INFO] [1676474926.331209739] [moveit_robot_model.robot_model]: Loading robot model 'lite6'...
[INFO] [1676474926.331221189] [moveit_robot_model.robot_model]: No root/virtual joint specified in SRDF. Assuming fixed joint
[INFO] [1676474926.423631045] [kdl_parser]: Link link_base had 1 children.
[INFO] [1676474926.423648755] [kdl_parser]: Link link1 had 1 children.
[INFO] [1676474926.423655855] [kdl_parser]: Link link2 had 1 children.
[INFO] [1676474926.423658695] [kdl_pars

## Motion Planning Example

In [3]:
lite6.set_start_state_to_current_state()

# set constraints message
joint_values = {
    "joint1": math.radians(100),
    "joint2": math.radians(10.4),
    "joint3": math.radians(31.1),
    "joint4": math.radians(-1.5),
    "joint5": math.radians(21.5),
    "joint6": math.radians(1.3),
}
robot_state = RobotState(moveit.get_robot_model())
robot_state.joint_positions = joint_values
joint_constraint = construct_joint_constraint(
    robot_state=robot_state,
    joint_model_group=moveit.get_robot_model().get_joint_model_group("lite6"),
)
lite6.set_goal_state(motion_plan_constraints=[joint_constraint])

# plan to goal
plan_result = lite6.plan()

# execute the plan
if plan_result:
    robot_trajectory = plan_result.trajectory
    moveit.execute(robot_trajectory, blocking=True, controllers=[])

[INFO] [1676474928.260037177] [moveit.ompl_planning.model_based_planning_context]: Planner configuration 'lite6' will use planner 'geometric::RRTConnect'. Additional configuration parameters will be set when the planner is constructed.
[INFO] [1676474930.286417091] [moveit.plugins.moveit_simple_controller_manager]: Returned 1 controllers in list
[INFO] [1676474930.286456821] [moveit.plugins.moveit_simple_controller_manager]: Returned 1 controllers in list
[INFO] [1676474930.286622902] [moveit.plugins.moveit_simple_controller_manager]: Returned 1 controllers in list
[INFO] [1676474930.286635962] [moveit.plugins.moveit_simple_controller_manager]: Returned 1 controllers in list
[INFO] [1676474930.286742713] [moveit_ros.trajectory_execution_manager]: Validating trajectory with allowed_start_tolerance 0.01
[INFO] [1676474930.379210879] [moveit_ros.trajectory_execution_manager]: Starting trajectory execution ...
[INFO] [1676474930.379252179] [moveit.plugins.moveit_simple_controller_manager]:

In [4]:
lite6.set_start_state_to_current_state()
lite6.set_goal_state(configuration_name="home")
# plan to goal
plan_result = lite6.plan()

# execute the plan
if plan_result:
    robot_trajectory = plan_result.trajectory
    moveit.execute(robot_trajectory, blocking=True, controllers=[])

[INFO] [1676474972.442897196] [moveit.ompl_planning.model_based_planning_context]: Planner configuration 'lite6' will use planner 'geometric::RRTConnect'. Additional configuration parameters will be set when the planner is constructed.
[INFO] [1676474974.337887947] [moveit.plugins.moveit_simple_controller_manager]: Returned 1 controllers in list
[INFO] [1676474974.337931837] [moveit.plugins.moveit_simple_controller_manager]: Returned 1 controllers in list
[INFO] [1676474974.338083938] [moveit.plugins.moveit_simple_controller_manager]: Returned 1 controllers in list
[INFO] [1676474974.338096548] [moveit.plugins.moveit_simple_controller_manager]: Returned 1 controllers in list
[INFO] [1676474974.338199119] [moveit_ros.trajectory_execution_manager]: Validating trajectory with allowed_start_tolerance 0.01
[INFO] [1676474974.379113635] [moveit_ros.trajectory_execution_manager]: Starting trajectory execution ...
[INFO] [1676474974.379147045] [moveit.plugins.moveit_simple_controller_manager]:

## Teleop

In [ ]:
from moveit.servo_client.devices.ps4_dualshock import PS4DualShockTeleop


ps4 = PS4DualShockTeleop()
ps4.start_teleop()